In [1]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
import os
import factor_analyzer
from factor_analyzer import FactorAnalyzer
pd.options.display.float_format = '{:.2f}'.format

/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#loading all the data
dfc=pd.read_csv('dfc.csv')
dfv=pd.read_csv('dfv.csv')
dfv_ColNames=pd.read_csv('dfv_ColNames.csv')
dfv_noDupe_ColNames=pd.read_csv('dfv_noDupe_ColNames.csv')
dfvnoDupe_recode=pd.read_csv('dfv_noDupe_recode.csv')
dfv_noDupe=pd.read_csv('dfv_noDupe.csv')
dfv_recode=pd.read_csv('dfv_recode.csv')
dfv_original=pd.read_csv('dfV_original.csv')
dfc_original=pd.read_csv('dfC_orignial.csv')

In [3]:
#add dfv variables into dfc
dfc.Case= dfc.Case.astype('category')
dfv.Case= dfv.Case.astype('category')
# merge dfc and selected columns from dfv_noDupe
dfc = pd.merge(dfc, dfv, on=['Case'], how='inner', indicator=True)

In [4]:
#testing Cronbach's alpha 1
trendiness = dfv[['Humor', 'Animation', 'Filters', 'Music']]
pg.cronbach_alpha(data=trendiness)

(0.7044743642050865, array([0.61 , 0.781]))

In [5]:
#testing Cronbach's alpha 2
trendinessAll = dfv_noDupe[['Shots', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing', 'Decorum','Language','Animation', 'Filters']]
pg.cronbach_alpha(data=trendinessAll)

(0.27718522477638524, array([0.077, 0.45 ]))

In [6]:
#testing Cronbach's alpha 3
trendiness = dfv_noDupe[['Music', 'Humor', 'Decorum','Language','Animation', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.37158439790693765, array([0.185, 0.527]))

In [7]:
#testing Cronbach's alpha 4
trendiness = dfv_noDupe[['Music', 'Humor', 'Decorum','Language', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.38564121023881226, array([0.198, 0.54 ]))

In [8]:
#testing Cronbach's alpha 5
trendiness = dfv_noDupe[['EditsStyle','Music', 'Humor', 'Decorum','Language', 'Filters']]
pg.cronbach_alpha(data=trendiness)

(0.37196059920590563, array([0.185, 0.528]))

In [9]:
#Running Factor Analysis since Cronbach's alpha is not high enough
fa=FactorAnalyzer(n_factors=6, rotation="varimax")#setting terms
fa.fit(trendinessAll)#applying terms for df with target variables
ev, v = fa.get_eigenvalues()#eigenvalues
EFAloaging=pd.DataFrame(fa.loadings_)#creating EFA loading df
EFAloaging.to_csv('EFAloaging.csv')#saving EFA loading df
EFAloaging

,0,1,2,3,4,5
0,0.08,-0.15,0.03,-0.01,-0.00,-0.58
1,0.11,0.17,0.71,-0.10,0.04,-0.18
2,-0.00,-0.39,0.69,-0.22,0.33,0.48
3,-0.13,-0.29,0.16,-0.32,0.04,0.17
4,-0.02,0.60,0.30,-0.04,0.02,0.22
5,0.20,0.13,0.53,0.11,0.17,0.07
6,0.09,0.33,0.33,-0.07,0.87,0.03
7,0.13,0.04,-0.00,0.99,0.04,-0.04
8,-0.06,-0.08,-0.01,-0.19,0.12,-0.12
9,-0.02,0.65,-0.06,0.13,0.05,0.04


In [10]:
pd.DataFrame(ev)#factor variance

,0
0,3.00
1,2.20
2,1.87
3,1.13
4,0.92
5,0.86
6,0.74
7,0.66
8,0.54
9,0.46


In [11]:
#some modification so I can read the EFA table
trendiness2 = trendinessAll.copy()
trendiness2.columns = [str(i) + '_' + col for i, col in enumerate(trendinessAll.columns)]
trendiness2.columns

Index(['0_Shots', '1_EditsStyle', '2_Broll', '3_NatSound', '4_Audio',
       '5_Music', '6_Narration', '7_Humor', '8_Sourcing', '9_Decorum',
       '10_Language', '11_Animation', '12_Filters'],
      dtype='object')

<h3>Cronbach's Alpha tested for the variables that were related in the factor analysis</h3>

In [12]:
#observed factors
factor0=['NatSound', 'Animation', 'Filters']
factor1=['Audio', 'Decorum', 'Language']
factor2=['EditsStyle', 'Broll', 'Music']
factor4=['Broll', 'Music', 'Narration']
factor10=['EditsStyle', 'Broll', 'Music','Narration']
factor11=['EditsStyle', 'Music','Narration', 'Humor']
factor12=['Humor', 'Decorum', 'Language']
#creating a factorlist
factorlist = [factor0, factor1, factor2, factor4, factor10, factor11, factor12]
for factor in factorlist:
    Cronbach = dfv_noDupe[factor]
    print(factor)
    print(pg.cronbach_alpha(data=Cronbach))

['NatSound', 'Animation', 'Filters']
(0.4239994564109538, array([0.225, 0.579]))
['Audio', 'Decorum', 'Language']
(0.4610983049323681, array([0.275, 0.606]))
['EditsStyle', 'Broll', 'Music']
(0.6545623836126631, array([0.535, 0.747]))
['Broll', 'Music', 'Narration']
(0.6612613487401078, array([0.544, 0.752]))
['EditsStyle', 'Broll', 'Music', 'Narration']
(0.7133694189793809, array([0.622, 0.787]))
['EditsStyle', 'Music', 'Narration', 'Humor']
(0.5034428458950686, array([0.345, 0.631]))
['Humor', 'Decorum', 'Language']
(0.4587102851961875, array([0.272, 0.604]))


<h3>Since the following has satisfactory Cronbach's Alpha, we can use the variables in the factor analysis to create a composite score for TRENDINESS<br /> 
<br /> ['EditsStyle', 'Broll', 'Music', 'Narration']
<br /> (0.7133694189793809, array([0.622, 0.787]))


In [13]:
#adding 'trendiness' variable to dfv and dfv_noDupe 

# Define the factors
factor0=['NatSound', 'Animation', 'Filters']
factor1=['Audio', 'Decorum', 'Language']
factor2=['EditsStyle', 'Broll', 'Music']
factor4=['Broll', 'Music', 'Narration']
factor10=['EditsStyle', 'Broll', 'Music','Narration']

# Create new variables in dfv
dfv['factor0'] = dfv[factor0].sum(axis=1)
dfv['factor1'] = dfv[factor1].sum(axis=1)
dfv['factor2'] = dfv[factor2].sum(axis=1)
dfv['factor4'] = dfv[factor4].sum(axis=1)
dfv['trendiness'] = dfv[factor10].sum(axis=1)/4

# Create new variable, factor 10 in dfv_noDupe, and dfc
dfv_noDupe['trendiness'] = dfv_noDupe[factor10].sum(axis=1)/4
dfc['trendiness'] = dfc[factor10].sum(axis=1)/4


<h1>Now to Hypothesis Testing</h1>

<h2>H1a: Trendiness is more prevalent on TikTok compared to YouTube Shorts and Instagram Reels.</h2>

In [14]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['trendiness']
DF = dfv
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,trendiness,Platform,2,123,0.97,0.38,0.02


In [15]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,trendiness,Platform,Instagram Reels,TikTok,False,True,1.11,82.00,two-sided,0.27,0.80,bonf,0.391,0.24
1,trendiness,Platform,Instagram Reels,YouTube Shorts,False,True,1.27,82.00,two-sided,0.21,0.63,bonf,0.457,0.27
2,trendiness,Platform,TikTok,YouTube Shorts,False,True,0.23,82.00,two-sided,0.82,1.00,bonf,0.233,0.05


In [16]:
mean_std_table

,Variable,Value,Mean,Standard Deviation
0,trendiness,Instagram Reels,0.64,0.35
1,trendiness,TikTok,0.56,0.33
2,trendiness,YouTube Shorts,0.54,0.38


<h2>H1b: Social media only news outlets have content that is trendier compared to that of hybrid news outlets.
<br />H1c: Hybrid news outlets have content that is trendier compared to that of legacy news outlets.</h2>

In [17]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['trendiness']
DF = dfv_noDupe
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova2 = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova2.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,trendiness,Platform,2,123,0.97,0.38,0.02


In [18]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,trendiness,MediaType,Hybrid,Legacy,False,True,-1.60,82.00,two-sided,0.11,0.34,bonf,0.692,-0.35
1,trendiness,MediaType,Hybrid,SMonly,False,True,-0.21,82.00,two-sided,0.84,1.00,bonf,0.232,-0.04
2,trendiness,MediaType,Legacy,SMonly,False,True,1.01,82.00,two-sided,0.31,0.94,bonf,0.356,0.22


In [19]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,trendiness,Hybrid,0.54,0.31
1,trendiness,Legacy,0.64,0.27
2,trendiness,SMonly,0.56,0.46


<h2>H2: Positive comment sentiment is higher on videos that have trendy presentations. (Comment sentiment is positively correlated to trendiness.)</h2>

In [20]:
pg.corr(dfc.trendiness, dfc.Sentiment, method='pearson').round(3)

,n,r,CI95%,p-val,BF10,power
pearson,979,-0.05,"[-0.12, 0.01]",0.10,0.161,0.39


<h2>H3a: Videos with traditional presentations have less comments.</h2>

In [21]:
pg.corr(dfv.trendiness, dfv.Comments, method='pearson').round(3)

,n,r,CI95%,p-val,BF10,power
pearson,126,0.02,"[-0.16, 0.19]",0.83,0.114,0.06


<h2>H4a: Comments overall are a more negative sentiment compared to neutral or positive.

In [22]:
dfc.Sentiment.value_counts()

Sentiment
3.00    360
2.00    296
4.00    162
1.00    111
5.00     50
Name: count, dtype: int64

<h2>H4b: Comments on TikTok are more positive compared to Instagram Reels and YouTube Shorts.</h2>

<h2>Revised H4a: Comment sentiment is different depending on the platform.
<br />
H4b: Sentiment is higher on TikTok compared to Instagram Reels.
<br />
H4c: Sentiment is higher on TikTok compared to YouTube Shorts.
<br />
H4d: Sentiment is higher on Instagram Reels compared to YouTube Shorts.

In [23]:
# Define the IV and DV variables
IV = 'Platform_x'
DV = ['Sentiment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
# merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
# merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Sentiment,Platform_x,2,976,5.36,0.00,0.01


In [24]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Sentiment,Platform_x,Instagram Reels,TikTok,False,True,-3.07,761.73,two-sided,0.00,0.01,bonf,8.014,-0.22
1,Sentiment,Platform_x,Instagram Reels,YouTube Shorts,False,True,-2.21,512.18,two-sided,0.03,0.08,bonf,1.004,-0.18
2,Sentiment,Platform_x,TikTok,YouTube Shorts,False,True,0.60,476.80,two-sided,0.55,1.00,bonf,0.114,0.05


In [25]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Sentiment,Instagram Reels,2.61,1.11
1,Sentiment,TikTok,2.84,0.99
2,Sentiment,YouTube Shorts,2.79,0.91


<h2>Report the following as descriptive statistics as the New RQ1:

<h3>Descriptive statistics per platform

In [27]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['Shots', 'Animation', 'Filters', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing']
DF = dfv
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# saving files
subfolder_path = IV+'/'
# Create the subfolder if it doesn't exist
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)

In [28]:
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Shots,Platform,2,120,0.10,0.91,0.00
1,Animation,Platform,2,123,0.04,0.96,0.00
2,Filters,Platform,2,123,0.04,0.96,0.00
3,EditsStyle,Platform,2,123,0.48,0.62,0.01
4,Broll,Platform,2,123,0.24,0.79,0.00
5,NatSound,Platform,2,123,0.13,0.88,0.00
6,Audio,Platform,2,123,0.19,0.83,0.00
7,Music,Platform,2,123,0.98,0.38,0.02
8,Narration,Platform,2,123,0.88,0.42,0.01
9,Humor,Platform,2,123,1.40,0.25,0.02


In [29]:
merged_pairwise[merged_pairwise['Variable'] == 'Offensive']

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
30,Offensive,Platform,Instagram Reels,TikTok,False,True,-1.13,82.00,two-sided,0.26,0.79,bonf,0.395,-0.24
31,Offensive,Platform,Instagram Reels,YouTube Shorts,False,True,2.00,82.00,two-sided,0.05,0.15,bonf,1.273,0.43
32,Offensive,Platform,TikTok,YouTube Shorts,False,True,2.54,82.00,two-sided,0.01,0.04,bonf,3.619,0.55


In [30]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Shots,Instagram Reels,0.77,0.43
1,Shots,TikTok,0.81,0.40
2,Shots,YouTube Shorts,0.79,0.42
3,Animation,Instagram Reels,0.69,0.47
4,Animation,TikTok,0.69,0.47
5,Animation,YouTube Shorts,0.71,0.46
6,Filters,Instagram Reels,0.69,0.47
7,Filters,TikTok,0.69,0.47
8,Filters,YouTube Shorts,0.71,0.46
9,EditsStyle,Instagram Reels,0.74,0.45


In [31]:
# Define the IV and DV variables
IV = 'Publisher'
DV = ['Shots', 'Animation', 'Filters', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Sourcing']
DF = dfv_noDupe
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
# saving files
subfolder_path = IV+'/'
# Create the subfolder if it doesn't exist
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)

/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/stephenwsong/anaconda3/e

In [32]:
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Shots,Publisher,5,120,1.48,0.20,0.06
1,Animation,Publisher,5,120,12.35,0.00,0.34
2,Filters,Publisher,5,120,12.35,0.00,0.34
3,EditsStyle,Publisher,5,120,12.55,0.00,0.34
4,Broll,Publisher,5,120,22.26,0.00,0.48
5,NatSound,Publisher,5,120,16.51,0.00,0.41
6,Audio,Publisher,5,120,2.63,0.03,0.10
7,Music,Publisher,5,120,21.07,0.00,0.47
8,Narration,Publisher,5,120,7.10,0.00,0.23
9,Humor,Publisher,5,120,14.57,0.00,0.38


In [33]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Shots,Publisher,CNN,GoodMorning BadNews,False,True,-3.16,40.00,two-sided,0.00,0.04,bonf,12.803,-0.96
1,Shots,Publisher,CNN,NPR,False,True,-1.04,40.00,two-sided,0.31,1.00,bonf,0.466,-0.31
2,Shots,Publisher,CNN,Tampa Bay Times,False,True,-0.33,40.00,two-sided,0.75,1.00,bonf,0.316,-0.10
3,Shots,Publisher,CNN,The Washington Post,False,True,-1.48,40.00,two-sided,0.15,1.00,bonf,0.719,-0.45
4,Shots,Publisher,CNN,UnderTheDeskNews,False,True,0.00,40.00,two-sided,1.00,1.00,bonf,0.303,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Sourcing,Publisher,NPR,The Washington Post,False,True,1.83,40.00,two-sided,0.08,1.00,bonf,1.121,0.55
176,Sourcing,Publisher,NPR,UnderTheDeskNews,False,True,2.50,40.00,two-sided,0.02,0.25,bonf,3.351,0.76
177,Sourcing,Publisher,Tampa Bay Times,The Washington Post,False,True,0.47,40.00,two-sided,0.64,1.00,bonf,0.331,0.14
178,Sourcing,Publisher,Tampa Bay Times,UnderTheDeskNews,False,True,1.24,40.00,two-sided,0.22,1.00,bonf,0.556,0.37


In [34]:
mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Sourcing,CNN,0.81,0.40
1,Sourcing,GoodMorning BadNews,0.95,0.22
2,Sourcing,NPR,1.00,0.00
3,Sourcing,Tampa Bay Times,0.90,0.30
4,Sourcing,The Washington Post,0.86,0.36
5,Sourcing,UnderTheDeskNews,0.76,0.44
